In [1]:
!pip install pysastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 7.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
import re
import numpy as np
import seaborn as sns
import warnings
import string
warnings.filterwarnings("ignore", 'This pattern has match groups')
import matplotlib.pyplot as plt
import matplotlib

In [3]:
version = 3
df_human = pd.read_csv(f"Dataset/Human_Annotated.csv", index_col="ID")
df_human.fillna("", inplace = True)

df_human_umumnt = df_human.drop(columns=["Umum"])
df_human_umumnt["COUNT CLASS"] = df_human_umumnt.drop(columns=["JUDUL", "ISI"]).sum(axis=1)
df_human = df_human_umumnt[df_human_umumnt["COUNT CLASS"] > 0].drop(columns=["COUNT CLASS"])
del df_human_umumnt

# Get from corpus

In [4]:
h_code = list(df_human.index)

tree = ET.parse('Raw_Corpus_Abid/Corpus.xml')
root = tree.getroot()
df_lst = []
for dok in root:
  lst = []
  for tag in dok:
    if tag.tag == "ID" and tag.text in h_code:
      lst.append(tag.text)
      h_code.remove(tag.text)
    elif tag.tag == "ID" and tag.text not in h_code:
      break
    elif tag.tag == "TAG":
      labels = []
      for label in tag:
        labels.append(label.text)
      lst.append(labels)
    elif tag.tag == "KELUHAN":
      lst.append(tag[0].text)
      lst.append(tag[1].text)
    elif tag.tag == "JAWABAN":
      lst.append(tag[0].text)
  if lst != []:
    df_lst.append(lst)
df_corpus = pd.DataFrame(data=df_lst, columns=["ID", "CLASS", "JUDUL", "ISI", "JAWABAN"]).set_index("ID")
del df_lst

# Improved Map


In [5]:
def mapping(s):
  temp = set()
  for key, val in map.items():
    if is_in(key.lower(), s.lower()):
      temp.update(val)
  return list(temp)

def is_in(kw, text):
  if (len(kw.split()) == 1):
    kw = " " + kw + " "
    text = " " + text + " "
  return kw in text

def mapping_2(s):
  res = {}
  for key, val in map.items():
    if is_in(key.lower(), s.lower()):
      for label in val:
        if label not in res:
          res[label] = 0
        res[label] += 1
  return get_top_3(res)

import statistics
def get_top_3(res):
    if len(res) == 0:
      return []
    elif len(res) == 1:
      return res.keys()
    max_value = max(res.values())
    std_dev = statistics.stdev(res.values())
    top = [(key, value) for key, value in res.items() if value >= (max_value - std_dev)]

    sorted_res = sorted(top, key=lambda x: x[1], reverse=True)

    # Get the top 3 keys
    return [item[0] for item in sorted_res[:3]]


In [6]:
map = {
    "Anak" : ["Anak"],
    "Anak Remaja" : ["Anak"],
    "Asi" : ["Kandungan"],
    "Laktasi" : ["Kandungan"],
    "Kandungan" : ["Kandungan"],
    "Fat Loss dan Fitness" : ["Gizi"],
    "Jantung" : ["Jantung"],
    "Psikiater" : ["Jiwa"],


    "Alergi Anak" : ["Anak"],
    "Gizi Anak" : ["Anak"],
    "Kebidanan Kandungan" : ["Kandungan"],    "Bidan Kandung" : ["Kandungan"],
    "Haid" : ["Kandungan"],
    "Bedah" : ["Bedah"],
    "Gigi Mulut" : ["Gigi"],
    "Gizi" : ["Gizi"],
    "Berat Badan Ideal" : ["Gizi"],
    "Kecantikan" : ["Kulit dan Kelamin"],     "Cantik" : ["Kulit dan Kelamin"],
    "Kulit Dan Kelamin" : ["Kulit dan Kelamin"],
    "Masalah Kulit" : ["Kulit dan Kelamin"],
    "Kesehatan Jiwa" : ["Jiwa"],    "Sehat Jiwa" : ["Jiwa"],
    "Psikiatri" : ["Jiwa"],
    "Mata" : ["Mata"],
    "Penyakit Dalam" : ["Penyakit Dalam"],    "Sakit Dalam" : ["Penyakit Dalam"],
    "Tht" : ["THT"],
    "Tulang" : ["Tulang"],

    "Flu Singapura" : ["THT"],
    "Influenza" : ["THT"],
    "Anemia" : ["Penyakit Dalam"],

    "Abses" : ["Bedah"],
    "Aids" : ["Penyakit Dalam"],
    "Alergi" : ["Kulit dan Kelamin"],
    "Alergi Dingin" : ["Kulit dan Kelamin"],
    "Alkohol" : ["Penyakit Dalam"],
    "Amandel" : ["THT"],
    "Amlodipine" : ["Penyakit Dalam"],
    "Anak" : ["Anak"],
    "Anemia Defisiensi Besi" : ["Penyakit Dalam"],
    "Angin Duduk" : ["Jantung"],
    "Artritis" : ["Penyakit Dalam"],
    "Asam Lambung" : ["Penyakit Dalam"],
    "Asam Urat" : ["Penyakit Dalam"],
    "Asma" : ["Anak", "Paru"],
    "Autisme" : ["Anak"],
    "Batu Empedu" : ["Bedah"],
    "Batu Ginjal" : ["Bedah"],
    "Batuk" : ["THT"],
    "Bau Mulut" : ["Penyakit Dalam", "Gigi"],
    "Bayi" : ["Anak"],
    "Bekas Luka" : ["Kulit dan Kelamin"],
    "Benjolan" : ["Bedah"],       "Benjol" : ["Bedah"],
    "Bersalin" : ["Kandungan"],   "Salin" : ["Kandungan"],
    "Biang Keringat" : ["Kulit dan Kelamin"],
    "Bibir" : ["Kulit dan Kelamin"],
    "Biduran" : ["Kulit dan Kelamin"],      "Bidur" : ["Kulit dan Kelamin"],
    "Bintitan" : ["Mata"],                  "Bintit" : ["Mata"],
    "Bipolar" : ["Jiwa"],
    "Bph" : ["Urologi", "Bedah"],
    "Bronkitis" : ["Paru"],
    "Buta Warna" : ["Mata"],
    "Cacar Air" : ["Kulit dan Kelamin"],
    "Campak" : ["Anak"],
    "Cedera" : ["Bedah"],
    "Dehidrasi" : ["Penyakit Dalam"],
    "Demam" : ["Penyakit Dalam"],
    "Demam Berdarah" : ["Penyakit Dalam"],      "Demam Darah" : ["Penyakit Dalam"],
    "Demam Pada Anak" : ["Anak"],
    "Depresi" : ["Jiwa"],
    "Dermatitis" : ["Kulit dan Kelamin"],
    "Diabetes" : ["Penyakit Dalam"],
    "Diabetes Tipe 1" : ["Penyakit Dalam"],
    "Diabetes Tipe 2" : ["Penyakit Dalam"],
    "Diare" : ["Penyakit Dalam"],
    "Diet" : ["Gizi"],
    "Disentri" : ["Penyakit Dalam"],
    "Eksim" : ["Kulit dan Kelamin"],
    "Endometriosis" : ["Kandungan"],
    "Flu" : ["THT"],
    "Gagal Ginjal" : ["Penyakit Dalam"],
    "Gagal Jantung" : ["Jantung"],
    "Gastritis" : ["Penyakit Dalam"],
    "Gastroenteritis" : ["Penyakit Dalam"],
    "Gatal" : ["Kulit dan Kelamin"],
    "Gigi" : ["Gigi"],
    "Glaukoma" : ["Mata"],
    "Gondok" : ["Penyakit Dalam"],
    "Gonore" : ["Kulit dan Kelamin"],
    "Hamil" : ["Kandungan"],
    "Hamil Anggur" : ["Kandungan"],
    "Hematuria" : ["Urologi", "Penyakit Dalam", "Kandungan"],
    "Hepatitis" : ["Penyakit Dalam"],
    "Hepatitis A" : ["Penyakit Dalam"],
    "Hepatitis B" : ["Penyakit Dalam"],
    "Hepatitis C" : ["Penyakit Dalam"],
    "Hernia" : ["Bedah"],
    "Herpes Genital" : ["Kulit dan Kelamin"],
    "Herpes Zoster" : ["Kulit dan Kelamin"],
    "Hidrosefalus" : ["Anak"],
    "Hipertensi" : ["Jantung"],
    "Hipertiroidisme" : ["Penyakit Dalam"],
    "Hipoglikemi" : ["Penyakit Dalam"],
    "Hipoglikemia" : ["Penyakit Dalam"],
    "Hipotensi" : ["Jantung"],
    "Hipotiroidisme" : ["Penyakit Dalam"],
    "Hiv" : ["Penyakit Dalam"],
    "Hpv" : ["Kandungan", "Kulit dan Kelamin"],
    "Impetigo" : ["Kulit dan Kelamin"],
    "Imunisasi" : ["Anak"],
    "Infeksi Gigi" : ["Gigi"],
    "Infeksi Ginjal" : ["Penyakit Dalam"],
    "Infeksi Jamur" : ["Kulit dan Kelamin"],
    "Infeksi Kulit" : ["Kulit dan Kelamin"],
    "Infeksi Mata" : ["Mata"],
    "Infeksi Rahim" : ["Kandungan"],
    "Infeksi Saluran Kemih" : ["Urologi"],    "Infeksi Salur Kemih" : ["Urologi"],
    "Infeksi Telinga" : ["THT"],
    "Infeksi Telinga Tengah" : ["THT"],
    "Infertilitas" : ["Kandungan"],
    "Insomnia" : ["Saraf", "Jiwa"],
    "Intim Laki-Laki" : ["Kulit dan Kelamin"],  "Intim Laki" : ["Kulit dan Kelamin"],
    "Ispa" : ["Paru", "Anak"],
    "Itp" : ["Penyakit Dalam"],
    "Jerawat" : ["Kulit dan Kelamin"],
    "Kanker Darah" : ["Penyakit Dalam", "Bedah"],
    "Kanker Kulit" : ["Kulit dan Kelamin"],
    "Kanker Paru-Paru" : ["Paru", "Bedah"],   "Kanker Paru" : ["Paru", "Bedah"],
    "Kanker Paru Paru" : ["Paru", "Bedah"],
    "Kanker Payudara" : ["Bedah", "Penyakit Dalam"],
    "Kanker Prostat" : ["Urologi", "Bedah"],
    "Kanker Rahim" : ["Kandungan", "Bedah"],
    "Kanker Serviks" : ["Bedah", "Kandungan"],
    "Kanker Tulang" : ["Tulang", "Bedah"],
    "Kanker Usus Besar" : ["Bedah", "Penyakit Dalam"],
    "Katarak" : ["Mata"],
    "Keguguran" : ["Kandungan"],        "Gugur" : ["Kandungan"],
    "Kehamilan" : ["Kandungan"],        "Hamil" : ["Kandungan"],
    "Kehamilan Ektopik" : ["Kandungan"],  "Hamil Ektopik" : ["Kandungan"],
    "Kelahiran" : ["Kandungan"],         "Lahir" : ["Kandungan"],
    "Kelenjar Getah Bening" : ["THT", "Bedah", "Penyakit Dalam"],
    "Keputihan" : ["Kandungan"],        "Putih" : ["Kandungan"],

    "Keringat" : ["Kulit dan Kelamin"],
    "Kesehatan Mental" : ["Jiwa"],      "Sehat Mental" : ["Jiwa"],
    "Keseleo" : ["Bedah"],
    "Ketombe" : ["Kulit dan Kelamin"],
    "Kista" : ["Kandungan"],
    "Kista Ovarium" : ["Kandungan"],
    "Kolesterol" : ["Jantung",  "Penyakit Dalam"],
    "Kolesterol Tinggi" : ["Jantung",  "Penyakit Dalam"],
    "Komedo" : ["Kulit dan Kelamin"],
    "Konstipasi" : ["Penyakit Dalam"],
    "Kontrasepsi" : ["Kandungan"],
    "Kram" : ["Tulang"],
    "Kudis" : ["Kulit dan Kelamin"],
    "Kulit" : ["Kulit dan Kelamin"],
    "Kulit Bersisik" : ["Kulit dan Kelamin"],     "Kulit Sisik" : ["Kulit dan Kelamin"],
    "Kurap" : ["Kulit dan Kelamin"],
    "Kutil" : ["Kulit dan Kelamin"],
    "Kutil Kelamin" : ["Kulit dan Kelamin"],
    "Kutu Rambut" : ["Kulit dan Kelamin"],
    "Lensa Kontak" : ["Mata"],
    "Lipoma" : ["Bedah"],
    "Luka Bakar" : ["Bedah"],
    "Lupus" : ["Penyakit Dalam"],
    "Maag" : ["Penyakit Dalam"],
    "Malaria" : ["Penyakit Dalam"],
    "Masturbasi" : ["Kulit dan Kelamin"],
    "Mata Merah" : ["Mata"],
    "Melahirkan" : ["Kandungan"],           "Lahir" : ["Kandungan"],
    "Menopause" : ["Kandungan"],
    "Menstruasi" : ["Kandungan"],
    "Menyusui" : ["Kandungan"],
    "Merencanakan Kehamilan" : ["Kandungan"],   "Rencana Hamil" : ["Kandungan"],
    "Merokok" : ["Paru"],         "Rokok" : ["Paru"],
    "Migrain" : ["Saraf"],
    "Mimisan" : ["THT"],
    "Miom" : ["Kandungan"],
    "Mual" : ["Penyakit Dalam"],
    "Muntah" : ["Penyakit Dalam"],
    "Nutrisi" : ["Gizi"],
    "Nyeri Dada" : ["Jantung"],
    "Nyeri Haid" : ["Kandungan"],


    "Nyeri Sendi" : ["Penyakit Dalam", "Tulang"],
    "Operasi" : ["Bedah"],
    "Orang Tua" : ["Penyakit Dalam"],
    "Osteoarthritis" : ["Tulang"],
    "Osteoporosis" : ["Tulang"],
    "Ovulasi" : ["Kandungan"],
    "Panu" : ["Kulit dan Kelamin"],
    "Pasangan" : ["Kandungan"],       "Pasang" : ["Kandungan"],
    "Patah Tulang" : ["Tulang"],
    "Payudara" : ["Bedah"],
    "Pemutihan Kulit" : ["Kulit dan Kelamin"],              "Putih Kulit" : ["Kulit dan Kelamin"],
    "Penyakit Ginjal" : ["Penyakit Dalam"],                 "Sakit Ginjal" : ["Penyakit Dalam"],
    "Penyakit Jantung" : ["Jantung"],                       "Sakit Jantung" : ["Jantung"],
    "Penyakit Kuning" : ["Penyakit Dalam"],                 "Sakit Kuning" : ["Penyakit Dalam"],
    "Penyakit Mata" : ["Mata"],                             "Sakit Mata" : ["Mata"],
    "Penyakit Menular Seksual" : ["Kulit dan Kelamin"],     "Sakit Tular Seksual" : ["Kulit dan Kelamin"],
    "Penyakit Radang Pinggul" : ["Tulang"],                 "Sakit Radang Pinggul" : ["Tulang"],
    "Perut Kembung" : ["Penyakit Dalam"],
    "Pilek" : ["THT"],
    "Pingsan" : ["Penyakit Dalam",  "Jantung", "Saraf"],
    "Plasenta Previa" : ["Kandungan"],
    "Pneumonia" : ["Anak", "Paru"],
    "Polip" : ["THT"],
    "Ppok" : ["Paru"],
    "Psoriasis" : ["Kulit dan Kelamin"],
    "Rabun Jauh" : ["Mata"],
    "Radang Gusi" : ["Gigi"],
    "Radang Usus" : ["Penyakit Dalam"],
    "Rambut" : ["Kulit dan Kelamin"],
    "Rambut Rontok" : ["Kulit dan Kelamin"],
    "Rematik Artritis" : ["Tulang"],
    "Rheumatoid Arthritis" : ["Tulang"],
    "Rhinitis" : ["THT"],
    "Ruam Kulit" : ["Kulit dan Kelamin"],
    "Rubella" : ["Penyakit Dalam"],
    "Sakit Gigi" : ["Gigi"],
    "Sakit Kaki" : ["Tulang"],
    "Sakit Lutut" : ["Tulang"],
    "Sakit Maag" : ["Penyakit Dalam"],
    "Sakit Perut" : ["Penyakit Dalam"],
    "Sakit Pinggang" : ["Penyakit Dalam"],
    "Sakit Telinga" : ["THT"],
    "Sakit Tenggorokan" : ["THT"],
    "Hubungan Intim" : ["Kandungan"],       "Hubung Intim" : ["Kandungan"],
    "Serangan Jantung" : ["Jantung"],       "Serang Jantung" : ["Jantung"],
    "Serangan Panik" : ["Jiwa"],            "Serang Panik" : ["Jiwa"],
    "Sesak Napas" : ["Paru"],
    "Sifilis" : ["Kulit dan Kelamin"],
    "Sinusitis" : ["THT"],
    "Sirosis" : ["Penyakit Dalam"],
    "Skizofrenia" : ["Jiwa"],
    "Skoliosis" : ["Tulang"],
    "Stres" : ["Jiwa"],
    "Sukralfat" : ["Penyakit Dalam"],
    "Tahi Lalat" : ["Kulit dan Kelamin"],
    "Telinga" : ["THT"],
    "Telinga Berdenging" : ["THT"],         "Telinga Denging" : ["THT"],
    "Tetanus" : ["Penyakit Dalam"],
    "Thalassemia" : ["Anak"],
    "Tifus" : ["Penyakit Dalam"],
    "Tonsilitis" : ["THT"],
    "Trigliserida" : ["Penyakit Dalam"],
    "Trikomoniasis" : ["Kulit dan Kelamin"],
    "Tuberkulosis" : ["Penyakit Dalam"],
    "Tukak Lambung" : ["Penyakit Dalam"],
    "Tumor" : ["Bedah"],
    "Tumor Otak" : ["Saraf", "Bedah"],
    "Tumor Payudara" : ["Bedah"],
    "Usus Buntu" : ["Bedah"],
    "Vagina" : ["Kulit dan Kelamin", "Kandungan"],
    "Vaginitis" : ["Kulit dan Kelamin"],
    "Vaginosis" : ["Kulit dan Kelamin"],
    "Vaksin" : ["Anak"],
    "Vaksinasi" : ["Anak"],
    "Varikokel" : ["Urologi", "Bedah"],
    "Varises" : ["Jantung"],
    "Vitamin A" : ["Mata"],
    "Vitamin C" : ["Kulit dan Kelamin"],
    "Vitamin E" : ["Kulit dan Kelamin"],
    "Vitiligo" : ["Kulit dan Kelamin"],
    "Wajah" : ["Penyakit Dalam"],
    "Wasir" : ["Bedah"],
}

# Cleaning Corpus

In [7]:
import re
df_corpus["JUDUL"] = [re.sub(r'[^\w]', ' ', str(s).lower()) for s in df_corpus["JUDUL"].values]
df_corpus["ISI"] = [re.sub(r'[^\w]', ' ', str(s).lower()) for s in df_corpus["ISI"].values]

In [8]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
factory = StopWordRemoverFactory()
# stopwords = factory.get_stop_words()
# ext_stopwords = ['dok', 'doc', 'dokter', 'terima', 'kasih', 'terimakasih']
# whitelist = ['kecil', 'besar', 'mata', 'masalah', 'belakang']
# all_stopwords = list((set(stopwords) | set(ext_stopwords)) - set(whitelist))
all_stopwords = ["di", 'pada', 'ingin', 'dan', 'yang', 'sangat', 'luar', 'biasa', 'bagi', 'bagian', 'saat', 'hanya', 'asa', 'terasa', 'saat', 'dengan', 'bikin', 'dari', 'dalam', 'sering', 'saya', 'untuk']
dictionary = ArrayDictionary(all_stopwords)
stopword = StopWordRemover(dictionary)
df_corpus["JUDUL"] = [stopword.remove(s) for s in df_corpus["JUDUL"]]
df_corpus["ISI"] = [stopword.remove(s) for s in df_corpus["ISI"]]

In [9]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
ignore = ["menyusui"]
df_corpus["JUDUL"] = [" ".join([stemmer.stem(w) if w not in ignore else w for w in str(s).split()]) for s in df_corpus["JUDUL"]]
df_corpus["ISI"] = [" ".join([stemmer.stem(w) if w not in ignore else w for w in str(s).split()]) for s in df_corpus["ISI"]]

In [10]:
df_corpus["CLASS_JUDUL"] = [mapping_2(s) for s in df_corpus["JUDUL"]]
df_corpus["CLASS_ISI"] = [mapping_2(s) for s in df_corpus["ISI"]]
df_corpus["CLASS_JUDUL_ISI"] = [mapping_2((j + " SEP " + i)) for j, i in df_corpus[["JUDUL", "ISI"]].values]
df_corpus["NEW_CLASS"] = [mapping_2(" , ".join(s)) for s in df_corpus["CLASS"]]

In [11]:
df_corpus.head(2)

,CLASS,JUDUL,ISI,JAWABAN,CLASS_JUDUL,CLASS_ISI,CLASS_JUDUL_ISI,NEW_CLASS
ID,,,,,,,,
DS-1,[other],mengapa keringat badan lebih,tanya konsultasi selamat malam dokter fathurro...,selamat malam Fathurrosi. Maaf berapa berat ba...,(Kulit dan Kelamin),(Kulit dan Kelamin),(Kulit dan Kelamin),[]
DS-23,[other],suka menghayal halusinasi,tanya konsultasi selamat siang dokter kali kha...,"Bila berkelanjutan, bisa saja memiliki ganggua...",[],[],[],[]


In [12]:
print("Jumlah instances CLASS_JUDUL yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL yang null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL"].str.len() == 0])))
print("Jumlah instances CLASS_ISI yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_ISI yang null: " + str(len(df_corpus[df_corpus["CLASS_ISI"].str.len() == 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL_ISI"].str.len() == 0])))

Jumlah instances CLASS_JUDUL yang bukan null: 1104
Jumlah instances CLASS_JUDUL yang null: 455
Jumlah instances CLASS_ISI yang bukan null: 1325
Jumlah instances CLASS_ISI yang null: 234
Jumlah instances CLASS_JUDUL_ISI yang bukan null: 1385
Jumlah instances CLASS_JUDUL_ISI yang null: 174


# Compare Metric

In [13]:
from sklearn.metrics import classification_report, accuracy_score, jaccard_score, \
hamming_loss, cohen_kappa_score, f1_score, precision_score, recall_score
import math
def eval_all(y_human, y_machine, mlb):
  print("================ Classification Report ================")
  print(classification_report(y_human, y_machine, target_names=mlb.classes_, digits=4))
  print("================ Multi Label Score ================")
  eval_multi_label(y_human, y_machine, mlb)
  print("================ Cohen Kappa ================")
  cohen_kappa(y_human, y_machine, mlb)

def eval_multi_label(y_test, prediction, mlb):
    # Butuh informasi lebih lengkap? silakan disimak di bawah ini
    print('Accuracy             :', accuracy_score(y_test, prediction))
    print('Precision            :', precision_score(y_test, prediction, average='micro'))
    print('Recall               :', recall_score(y_test, prediction, average='micro'))
    print('F1 Score             :', f1_score(y_test, prediction, average='micro'))
    print('Hamming Loss         :', hamming_loss(y_test, prediction))
    print('Jaccard Score')
    js = jaccard_score(y_test, prediction, average=None)
    for i, j in zip(mlb.classes_, js):
      print(f"{i:<20} : {j}")
    print('Jaccard Score Macro Average:', jaccard_score(y_test, prediction, average='macro'))
    print('Jaccard Score Micro Average:', jaccard_score(y_test, prediction, average='micro'))

def cohen_kappa(y_human, y_test, mlb):
  sum = 0
  count = 0
  for i in range(len(mlb.classes_)):
    score = cohen_kappa_score(y_human[:,i], y_test[:,i])
    if not math.isnan(score):
      sum += score
      count += 1
    print(f"Cohen Kappa {mlb.classes_[i]:<20} : {cohen_kappa_score(y_human[:,i], y_test[:,i])}")
  print("Average Cohen Kappa              : " + str(sum/count))

## By Metadata

In [14]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_ISI", "CLASS_JUDUL_ISI", "CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
class_col = "NEW_CLASS"
df = df[df['NEW_CLASS'].map(lambda d: len(d)) > 0]
df

,NEW_CLASS,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
AD-47,(Kandungan),0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
AD-117,"[Bedah, Penyakit Dalam]",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
AD-148,(Kulit dan Kelamin),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
AD-179,[Kandungan],0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
AD-233,[Jantung],0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43177,(Penyakit Dalam),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,(THT),0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
KD-43201,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb_std = MultiLabelBinarizer()
mlb_std.fit(df[class_col].values)

MultiLabelBinarizer()

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [17]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.8095    0.7907    0.8000       129
            Bedah     0.5000    0.5351    0.5169       114
             Gigi     0.9643    0.8438    0.9000        32
             Gizi     0.7361    0.5354    0.6199        99
          Jantung     0.1613    0.2273    0.1887        22
             Jiwa     0.8400    0.7412    0.7875        85
        Kandungan     0.8889    0.8333    0.8602       240
Kulit dan Kelamin     0.8171    0.8462    0.8314       169
             Mata     1.0000    0.8986    0.9466        69
             Paru     0.5625    0.5000    0.5294        18
   Penyakit Dalam     0.5560    0.6943    0.6175       193
            Saraf     0.5000    0.1163    0.1887        43
              THT     0.9098    0.8222    0.8638       135
           Tulang     0.6216    0.5610    0.5897        41
          Urologi     0.3333    0.0606    0.1026        33

## By Judul

In [18]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_ISI", "CLASS_JUDUL_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_JUDUL"].str.len() > 0]
class_col = "CLASS_JUDUL"
df

,CLASS_JUDUL,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,(Kulit dan Kelamin),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-176,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-324,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
AD-117,(Kandungan),0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
AD-148,(Kulit dan Kelamin),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43064,(Kulit dan Kelamin),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
KD-43177,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,(THT),0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [19]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [20]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.7500    0.7362    0.7430       163
            Bedah     0.5027    0.7815    0.6118       119
             Gigi     0.9615    0.8929    0.9259        28
             Gizi     0.8000    0.1818    0.2963        44
          Jantung     0.4426    0.9000    0.5934        30
             Jiwa     0.9697    0.5079    0.6667        63
        Kandungan     0.8052    0.8304    0.8176       224
Kulit dan Kelamin     0.6812    0.8525    0.7573       183
             Mata     0.9155    0.9848    0.9489        66
             Paru     0.6000    0.4615    0.5217        13
   Penyakit Dalam     0.5133    0.4715    0.4915       123
            Saraf     0.0000    0.0000    0.0000        25
              THT     0.8889    0.8889    0.8889       126
           Tulang     0.6562    0.6364    0.6462        33
          Urologi     0.5000    0.0500    0.0909        20

## By Isi

In [21]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_JUDUL_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_ISI"].str.len() > 0]
class_col = "CLASS_ISI"
df

,CLASS_ISI,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,(Kulit dan Kelamin),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-176,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-277,"[Saraf, Jiwa, Paru]",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
DS-324,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
AD-27,(Anak),0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43064,"[Kulit dan Kelamin, Penyakit Dalam]",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
KD-43177,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,"[Jantung, THT]",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [22]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [23]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.6756    0.8492    0.7525       179
            Bedah     0.4486    0.7559    0.5630       127
             Gigi     0.8889    0.9143    0.9014        35
             Gizi     0.5882    0.2778    0.3774        72
          Jantung     0.3333    0.8824    0.4839        34
             Jiwa     0.8444    0.4634    0.5984        82
        Kandungan     0.6905    0.8689    0.7695       267
Kulit dan Kelamin     0.6813    0.8906    0.7720       192
             Mata     0.7640    0.9315    0.8395        73
             Paru     0.6000    0.3158    0.4138        19
   Penyakit Dalam     0.4905    0.5421    0.5150       190
            Saraf     0.0000    0.0000    0.0000        44
              THT     0.8299    0.8414    0.8356       145
           Tulang     0.4737    0.6279    0.5400        43
          Urologi     0.0000    0.0000    0.0000        29

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## By Judul + Isi

In [24]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_JUDUL_ISI"].str.len() > 0]
class_col = "CLASS_JUDUL_ISI"
df

,CLASS_JUDUL_ISI,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,(Kulit dan Kelamin),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-176,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-277,"[Saraf, Jiwa, Paru]",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
DS-324,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
AD-27,(Anak),0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43064,"[Kulit dan Kelamin, Penyakit Dalam]",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
KD-43177,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,[THT],0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [25]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [26]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.6920    0.8962    0.7810       183
            Bedah     0.4489    0.7481    0.5611       135
             Gigi     0.8919    0.9167    0.9041        36
             Gizi     0.6757    0.3378    0.4505        74
          Jantung     0.3617    0.9189    0.5191        37
             Jiwa     0.8600    0.4886    0.6232        88
        Kandungan     0.7135    0.8809    0.7884       277
Kulit dan Kelamin     0.6856    0.9005    0.7785       201
             Mata     0.7826    0.9351    0.8521        77
             Paru     0.5833    0.3500    0.4375        20
   Penyakit Dalam     0.4889    0.5500    0.5176       200
            Saraf     0.0000    0.0000    0.0000        45
              THT     0.8366    0.8591    0.8477       149
           Tulang     0.4828    0.6087    0.5385        46
          Urologi     1.0000    0.0312    0.0606        32

# V2 Mapping

## Map2

In [27]:
map = {
    "Anak" : ["Anak"],
    "Anak Remaja" : ["Anak"],
    "Asi" : ["Kandungan"],
    "Laktasi" : ["Kandungan"],
    "Kandungan" : ["Kandungan"],
    "Fat Loss dan Fitness" : ["Gizi"],
    "Jantung" : ["Jantung"],
    "Psikiater" : ["Jiwa"],


    "Alergi Anak" : ["Anak"],
    "Gizi Anak" : ["Anak"],
    "Kebidanan Kandungan" : ["Kandungan"],    "Bidan Kandung" : ["Kandungan"],
    "Haid" : ["Kandungan"],
    "Bedah" : ["Bedah"],
    "Gigi Mulut" : ["Gigi"],
    "Gizi" : ["Gizi"],
    "Berat Badan Ideal" : ["Gizi"],
    "Kecantikan" : ["Kulit dan Kelamin"],     "Cantik" : ["Kulit dan Kelamin"],
    "Kulit Dan Kelamin" : ["Kulit dan Kelamin"],
    "Masalah Kulit" : ["Kulit dan Kelamin"],
    "Kesehatan Jiwa" : ["Jiwa"],    "Sehat Jiwa" : ["Jiwa"],
    "Psikiatri" : ["Jiwa"],
    "Mata" : ["Mata"],
    "Penyakit Dalam" : ["Penyakit Dalam"],    "Sakit Dalam" : ["Penyakit Dalam"],
    "Tht" : ["THT"],
    "Tulang" : ["Tulang"],

    "Flu Singapura" : ["THT"],
    "Influenza" : ["THT"],
    "Anemia" : ["Penyakit Dalam"],

    "Abses" : ["Bedah"],
    "Aids" : ["Penyakit Dalam"],
    "Alergi" : ["Kulit dan Kelamin"],
    "Alergi Dingin" : ["Kulit dan Kelamin"],
    "Alkohol" : ["Penyakit Dalam"],
    "Amandel" : ["THT"],
    "Amlodipine" : ["Penyakit Dalam"],
    "Anak" : ["Anak"],
    "Anemia Defisiensi Besi" : ["Penyakit Dalam"],
    "Angin Duduk" : ["Jantung"],
    "Artritis" : ["Penyakit Dalam"],
    "Asam Lambung" : ["Penyakit Dalam"],
    "Asam Urat" : ["Penyakit Dalam"],
    "Asma" : ["Anak", "Paru"],
    "Autisme" : ["Anak"],
    "Batu Empedu" : ["Bedah"],
    "Batu Ginjal" : ["Bedah"],
    "Batuk" : ["THT"],
    "Bau Mulut" : ["Penyakit Dalam", "Gigi"],
    "Bayi" : ["Anak"],
    "Bekas Luka" : ["Kulit dan Kelamin"],
    "Benjolan" : ["Bedah"],       "Benjol" : ["Bedah"],
    "Bersalin" : ["Kandungan"],   "Salin" : ["Kandungan"],
    "Biang Keringat" : ["Kulit dan Kelamin"],
    "Bibir" : ["Kulit dan Kelamin"],
    "Biduran" : ["Kulit dan Kelamin"],      "Bidur" : ["Kulit dan Kelamin"],
    "Bintitan" : ["Mata"],                  "Bintit" : ["Mata"],
    "Bipolar" : ["Jiwa"],
    "Bph" : ["Urologi", "Bedah"],
    "Bronkitis" : ["Paru"],
    "Bronchitis"  : ["Paru"],
    "Buta Warna" : ["Mata"],
    "Cacar Air" : ["Kulit dan Kelamin"],
    "Campak" : ["Anak"],
    "Cedera" : ["Bedah"],
    "Dehidrasi" : ["Penyakit Dalam"],
    "Demam" : ["Penyakit Dalam"],
    "Demam Berdarah" : ["Penyakit Dalam"],      "Demam Darah" : ["Penyakit Dalam"],
    "Demam Pada Anak" : ["Anak"],
    "Depresi" : ["Jiwa"],
    "Dermatitis" : ["Kulit dan Kelamin"],
    "Diabetes" : ["Penyakit Dalam"],
    "Diabetes Tipe 1" : ["Penyakit Dalam"],
    "Diabetes Tipe 2" : ["Penyakit Dalam"],
    "Diare" : ["Penyakit Dalam"],
    "Diet" : ["Gizi"],
    "Disentri" : ["Penyakit Dalam"],
    "Eksim" : ["Kulit dan Kelamin"],
    "Endometriosis" : ["Kandungan"],
    "Flu" : ["THT"],
    "Gagal Ginjal" : ["Penyakit Dalam"],
    "Gagal Jantung" : ["Jantung"],
    "Gastritis" : ["Penyakit Dalam"],
    "Gastroenteritis" : ["Penyakit Dalam"],
    "Gatal" : ["Kulit dan Kelamin"],
    "Gigi" : ["Gigi"],
    "Glaukoma" : ["Mata"],
    "Gondok" : ["Penyakit Dalam"],
    "Gonore" : ["Kulit dan Kelamin"],
    "Hamil" : ["Kandungan"],
    "Hamil Anggur" : ["Kandungan"],
    "Hematuria" : ["Urologi", "Penyakit Dalam", "Kandungan"],
    "Hepatitis" : ["Penyakit Dalam"],
    "Hepatitis A" : ["Penyakit Dalam"],
    "Hepatitis B" : ["Penyakit Dalam"],
    "Hepatitis C" : ["Penyakit Dalam"],
    "Hernia" : ["Bedah"],
    "Herpes Genital" : ["Kulit dan Kelamin"],
    "Herpes Zoster" : ["Kulit dan Kelamin"],
    "Hidrosefalus" : ["Anak"],
    "Hipertensi" : ["Jantung"],
    "Darah Tinggi" : ["Jantung"],
    "Hipertiroidisme" : ["Penyakit Dalam"],
    "Hipoglikemi" : ["Penyakit Dalam"],
    "Hipoglikemia" : ["Penyakit Dalam"],
    "Hipotensi" : ["Jantung"],
    "Darah Rendah" : ["Jantung"],
    "Hipotiroidisme" : ["Penyakit Dalam"],
    "Hiv" : ["Penyakit Dalam"],
    "Hpv" : ["Kandungan", "Kulit dan Kelamin"],
    "Impetigo" : ["Kulit dan Kelamin"],
    "Imunisasi" : ["Anak"],
    "Infeksi Gigi" : ["Gigi"],
    "Infeksi Ginjal" : ["Penyakit Dalam"],
    "Infeksi Jamur" : ["Kulit dan Kelamin"],
    "Infeksi Kulit" : ["Kulit dan Kelamin"],
    "Infeksi Mata" : ["Mata"],
    "Infeksi Rahim" : ["Kandungan"],
    "Infeksi Saluran Kemih" : ["Urologi"],    "Infeksi Salur Kemih" : ["Urologi"],
    "Infeksi Telinga" : ["THT"],
    "Infeksi Telinga Tengah" : ["THT"],
    "Infertilitas" : ["Kandungan"],
    "Insomnia" : ["Saraf", "Jiwa"],
    "Intim Laki-Laki" : ["Kulit dan Kelamin"],  "Intim Laki" : ["Kulit dan Kelamin"],
    "Ispa" : ["Paru", "Anak"],
    "Itp" : ["Penyakit Dalam"],
    "Jerawat" : ["Kulit dan Kelamin"],
    "Kanker Darah" : ["Penyakit Dalam", "Bedah"],
    "Kanker Kulit" : ["Kulit dan Kelamin"],
    "Kanker Paru-Paru" : ["Paru", "Bedah"],   "Kanker Paru" : ["Paru", "Bedah"],
    "Kanker Paru Paru" : ["Paru", "Bedah"],
    "Kanker Payudara" : ["Bedah", "Penyakit Dalam"],
    "Kanker Prostat" : ["Urologi", "Bedah"],
    "Kanker Rahim" : ["Kandungan", "Bedah"],
    "Kanker Serviks" : ["Bedah", "Kandungan"],
    "Kanker Tulang" : ["Tulang", "Bedah"],
    "Kanker Usus Besar" : ["Bedah", "Penyakit Dalam"],
    "Katarak" : ["Mata"],
    "Keguguran" : ["Kandungan"],        "Gugur" : ["Kandungan"],
    "Kehamilan" : ["Kandungan"],        "Hamil" : ["Kandungan"],
    "Kehamilan Ektopik" : ["Kandungan"],  "Hamil Ektopik" : ["Kandungan"],
    "Kelahiran" : ["Kandungan"],         "Lahir" : ["Kandungan"],
    "Kelenjar Getah Bening" : ["THT", "Bedah", "Penyakit Dalam"],
    "Keputihan" : ["Kandungan"],        "Putih" : ["Kandungan"],

    "Keringat" : ["Kulit dan Kelamin"],
    "Kesehatan Mental" : ["Jiwa"],      "Sehat Mental" : ["Jiwa"],
    "Keseleo" : ["Bedah"],
    "Ketombe" : ["Kulit dan Kelamin"],
    "Kista" : ["Kandungan"],
    "Kista Ovarium" : ["Kandungan"],
    "Kolesterol" : ["Jantung", "Penyakit Dalam"],
    "Kolesterol Tinggi" : ["Jantung", "Penyakit Dalam"],
    "Komedo" : ["Kulit dan Kelamin"],
    "Konstipasi" : ["Penyakit Dalam"],
    "Kontrasepsi" : ["Kandungan"],
    "Kram" : ["Tulang"],
    "Kudis" : ["Kulit dan Kelamin"],
    "Kulit" : ["Kulit dan Kelamin"],
    "Kulit Bersisik" : ["Kulit dan Kelamin"],     "Kulit Sisik" : ["Kulit dan Kelamin"],
    "Kurap" : ["Kulit dan Kelamin"],
    "Kutil" : ["Kulit dan Kelamin"],
    "Kutil Kelamin" : ["Kulit dan Kelamin"],
    "Kutu Rambut" : ["Kulit dan Kelamin"],
    "Lensa Kontak" : ["Mata"],
    "Lipoma" : ["Bedah"],
    "Luka Bakar" : ["Bedah"],
    "Lupus" : ["Penyakit Dalam"],
    "Maag" : ["Penyakit Dalam"],
    "Malaria" : ["Penyakit Dalam"],
    "Masturbasi" : ["Kulit dan Kelamin"],
    "Mata Merah" : ["Mata"],
    "Melahirkan" : ["Kandungan"],           "Lahir" : ["Kandungan"],
    "Menopause" : ["Kandungan"],
    "Menstruasi" : ["Kandungan"],
    "Menyusui" : ["Kandungan"],
    "Merencanakan Kehamilan" : ["Kandungan"],   "Rencana Hamil" : ["Kandungan"],
    "Merokok" : ["Paru"],         "Rokok" : ["Paru"],
    "Migrain" : ["Saraf"],
    "Mimisan" : ["THT"],
    "Miom" : ["Kandungan"],
    "Mual" : ["Penyakit Dalam"],
    "Muntah" : ["Penyakit Dalam"],
    "Nutrisi" : ["Gizi"],
    "Nyeri Dada" : ["Jantung"],
    "Nyeri Haid" : ["Kandungan"],


    "Nyeri Sendi" : ["Penyakit Dalam", "Tulang"],
    "Operasi" : ["Bedah"],
    "Orang Tua" : ["Penyakit Dalam"],
    "Osteoarthritis" : ["Tulang"],
    "Osteoporosis" : ["Tulang"],
    "Ovulasi" : ["Kandungan"],
    "Panu" : ["Kulit dan Kelamin"],
    "Pasangan" : ["Kandungan"],       "Pasang" : ["Kandungan"],
    "Patah Tulang" : ["Tulang"],
    "Payudara" : ["Bedah"],
    "Pemutihan Kulit" : ["Kulit dan Kelamin"],              "Putih Kulit" : ["Kulit dan Kelamin"],
    "Penyakit Ginjal" : ["Penyakit Dalam"],                 "Sakit Ginjal" : ["Penyakit Dalam"],
    "Penyakit Jantung" : ["Jantung"],                       "Sakit Jantung" : ["Jantung"],
    "Penyakit Kuning" : ["Penyakit Dalam"],                 "Sakit Kuning" : ["Penyakit Dalam"],
    "Penyakit Mata" : ["Mata"],                             "Sakit Mata" : ["Mata"],
    "Penyakit Menular Seksual" : ["Kulit dan Kelamin"],     "Sakit Tular Seksual" : ["Kulit dan Kelamin"],
    "Penyakit Radang Pinggul" : ["Tulang"],                 "Sakit Radang Pinggul" : ["Tulang"],
    "Perut Kembung" : ["Penyakit Dalam"],
    "Pilek" : ["THT"],
    "Pingsan" : ["Penyakit Dalam", "Jantung", "Saraf"],
    "Plasenta Previa" : ["Kandungan"],
    "Pneumonia" : ["Anak", "Paru"],
    "Polip" : ["THT"],
    "Ppok" : ["Paru"],
    "Psoriasis" : ["Kulit dan Kelamin"],
    "Rabun Jauh" : ["Mata"],
    "Radang Gusi" : ["Gigi"],
    "Radang Usus" : ["Penyakit Dalam"],
    "Rambut" : ["Kulit dan Kelamin"],
    "Rambut Rontok" : ["Kulit dan Kelamin"],
    "Rematik Artritis" : ["Tulang"],
    "Rheumatoid Arthritis" : ["Tulang"],
    "Rhinitis" : ["THT"],
    "Ruam Kulit" : ["Kulit dan Kelamin"],
    "Rubella" : ["Penyakit Dalam"],
    "Sakit Gigi" : ["Gigi"],
    "Sakit Kaki" : ["Tulang"],
    "Sakit Telapak Kaki" : ["Tulang"],
    "Sakit Lutut" : ["Tulang"],
    "Sakit Maag" : ["Penyakit Dalam"],
    "Sakit Lambung" : ["Penyakit Dalam"],
    "Ngilu Pusar" : ["Penyakit Dalam"],
    "Sakit Perut" : ["Penyakit Dalam"],
    "Sakit Pinggang" : ["Penyakit Dalam"],
    "Sakit Pingang" : ["Penyakit Dalam"],
    "Sakit Telinga" : ["THT"],
    "Sakit Tenggorokan" : ["THT"],
    "Hubungan Intim" : ["Kandungan"],       "Hubung Intim" : ["Kandungan"],
    "Serangan Jantung" : ["Jantung"],       "Serang Jantung" : ["Jantung"],
    "Serangan Panik" : ["Jiwa"],            "Serang Panik" : ["Jiwa"],
    "Sesak Napas" : ["Paru"],
    "Sesak Nafas" : ["Paru"],
    "Sifilis" : ["Kulit dan Kelamin"],
    "Sinusitis" : ["THT"],
    "Sirosis" : ["Penyakit Dalam"],
    "Skizofrenia" : ["Jiwa"],
    "Skoliosis" : ["Tulang"],
    "Stres" : ["Jiwa"],
    "Sukralfat" : ["Penyakit Dalam"],
    "Tahi Lalat" : ["Kulit dan Kelamin"],
    "Telinga" : ["THT"],
    "Telinga Berdenging" : ["THT"],         "Telinga Denging" : ["THT"],
    "Tetanus" : ["Penyakit Dalam"],
    "Thalassemia" : ["Anak"],
    "Tifus" : ["Penyakit Dalam"],
    "Tonsilitis" : ["THT"],
    "Trigliserida" : ["Penyakit Dalam"],
    "Trikomoniasis" : ["Kulit dan Kelamin"],
    "Tuberkulosis" : ["Penyakit Dalam"],
    "Tukak Lambung" : ["Penyakit Dalam"],
    "Tumor" : ["Bedah"],
    "Tumor Otak" : ["Saraf", "Bedah"],
    "Tumor Payudara" : ["Bedah"],
    "Usus Buntu" : ["Bedah"],
    "Vagina" : ["Kulit dan Kelamin", "Kandungan"],
    "Vaginitis" : ["Kulit dan Kelamin"],
    "Vaginosis" : ["Kulit dan Kelamin"],
    "Vaksin" : ["Anak"],
    "Vaksinasi" : ["Anak"],
    "Varikokel" : ["Urologi", "Bedah"],
    "Varises" : ["Jantung"],
    "Vitamin A" : ["Mata"],
    "Vitamin C" : ["Kulit dan Kelamin"],
    "Vitamin E" : ["Kulit dan Kelamin"],
    "Vitiligo" : ["Kulit dan Kelamin"],
    "Wajah" : ["Penyakit Dalam"],
    "Wasir" : ["Bedah"],

    'Alzheimer' : ['Saraf', 'Jiwa'],

    'Genetika' : ["Kandungan"],
    'Infeksi' : ["Penyakit Dalam"],
    'Intim' : ['Kandungan'],
    'Intim Laki Laki' : ["Kulit dan Kelamin", "Urologi"],
    'Kanker' : ["Bedah", 'Penyakit Dalam'],
    'Kejang' : ["Saraf"],
    'Meningitis' : ["Penyakit Dalam", "Saraf"],
    'Neuropati' : ['Saraf'],

    'Obesitas' : ['Gizi'],
    'Onkologi' : ["Bedah", 'Penyakit Dalam'],
    'Peningkatan Berat Badan' : ['Gizi'],         'Tingkat Berat Badan' : ['Gizi'],
    'Psikologi Seks Dan Perkawinan' : ['Jiwa'],   'Psikologi Seks Dan Kawin' : ['Jiwa'],
    'Pusing' : ['Saraf'],
    'Radang Otak' : ['Saraf'],
    'Sakit Kepala Tegang' : ['Saraf'],
    'Sariawan' : ['Gigi'],
    'Seks' : ['Kandungan', "Kulit dan Kelamin"],
    'Stroke' : ['Saraf', 'Penyakit Dalam'],
    'Suplemen' : ['Gizi'],
    'Vertigo' : ['THT', 'Saraf'],

    'Acyclovir' : ["Kulit dan Kelamin"],
    'Asam Mefenamat' : ["Gigi", "Kandungan"],

    'Bells Palsy' : ["Saraf"],
    'Cefadroxil' : ["THT", "Urologi", "Kandungan", "Kulit dan Kelamin", "Tulang", "Paru",  "Anak"],
    'Chlamydia' : ["Kulit dan Kelamin"],
    'Ciprofloxacin' : ["Paru", "THT", "Urologi", "Kulit dan Kelamin", "Penyakit Dalam", "Tulang"],
    'Clindamycin' : ["Kulit dan Kelamin", "Kandungan", "Paru", "Tulang", "Penyakit Dalam", "Bedah"],
    'Dexamethasone' : ["Penyakit Dalam", "Tulang", "Kulit dan Kelamin"],

    'Hormon' : ["Kandungan"],
    'Ibuprofen' : ["Gigi", "Tulang", "Kandungan", "Bedah", "Saraf", "Anak"],
    'Impotensi' : ["Urologi"],
    'Kanker Otak' : ["Bedah", 'Saraf'],
    'Kangker Otak' : ["Bedah", 'Saraf'],

    'Kesemutan' : ["Saraf"],          'Semut' : ["Saraf"],
    'Ketoconazole' :  ["Kulit dan Kelamin", "Penyakit Dalam"],
    'Lansoprazole' : ["Penyakit Dalam"],
    'Miconazole' : ["Kulit dan Kelamin"],

    'Omeprazole' : ["Penyakit Dalam"],
    'Paracetamol' : ["Anak", "Kandungan", "Tulang", "Gigi"],
    'Rabies' : ["Penyakit Dalam", "Saraf"],
    'Sakit Leher' : ["Saraf"],
    'Salbutamol' : ["Paru"],



    'Halusinasi'  : ["Jiwa"],
    'Combivent' : ["Paru"] ,
    'Sperma'  : ["Kandungan", "Urologi"],
    'Tenggorokan' : ["THT"],      'Tenggorok' : ["THT"],
    'Bunuh Diri'  : ["Jiwa"],
    'Metabolisme' : ["Gizi"],


    'Kacamata'  : ["Mata"],
    'Susu'  : ["Gizi", "Kandungan"],
    'Bisul' : ["Kulit dan Kelamin"],
    'Sunat' : ["Kulit dan Kelamin"],


    'Nystatin'  : ["Kulit dan Kelamin", "THT", "Gigi"],

    'Berat Badan' : ["Gizi"],
    "Perawatan Muka"  : ["Kulit dan Kelamin"],        "Awat Muka"  : ["Kulit dan Kelamin"],
    "Krim Pagi"  : ["Kulit dan Kelamin"],
    "Krim Malam"  : ["Kulit dan Kelamin"],
    "Prostat" : ["Urologi"],
    "Kadaluarsa"  : ["Gizi"],
    "Sinus" : ["THT"],
    "Hidung Tersumbat"  : ["THT"],            "Hidung Sumbat"  : ["THT"],
    'Rahim' : ["Kandungan"],
    'Pita Suara'  : ["THT"],
    'Tekanan Darah' : ["Jantung", "Penyakit Dalam"],    'Tekan Darah' : ["Jantung", "Penyakit Dalam"],
    'Urin'  : ["Urologi"],
    "Liver" : ["Penyakit Dalam"],
    "Feses" : ["Penyakit Dalam"],
    "Peredaran Darah"  : ["Jantung"],           "Edar Darah"  : ["Jantung"],
    "Flek"  : ["Kulit dan Kelamin", "Kandungan"],
    "Anyang Anyang" : ["Urologi", "Penyakit Dalam"],
    "Ayang" : ["Urologi", "Penyakit Dalam"],
    "Buang Air Kecil" : ["Urologi"],
    "Buang Air Besar" : ["Penyakit Dalam"],
    "Sakit Dada": ["Paru", 'Jantung'],
    "Masalah Jiwa"  : ["Jiwa"],
    "Ganggu Jiwa"  : ["Jiwa"],
    "Endoscopy" : ["Bedah", "Penyakit Dalam"],  "Endoskopi" : ["Bedah", "Penyakit Dalam"],
    "Kecil Paha"  : ["Gizi"],
    "Buncit"  : ["Gizi"],
    "Sakit Kepala" : ["Saraf"],
    "Ansietas"  : ["Jiwa"],   "Cemas" : ["Jiwa"],   "Anxiety" : ["Jiwa"],
    "Bab" : ["Penyakit Dalam"],
    "Muda Marah"  : ["Jiwa"],
    "Belakang Mental"  : ["Jiwa"],
    "Inseminasi"  : ["Kandungan"],
    "Kaki Dingin" : ["Jiwa"],
    "Gemuk" : ["Gizi"],
    "Cacar" : ["Kulit dan Kelamin"],
    "Selalu Kencing" : ["Urologi"],
    'Bunions' : ["Bedah", "Tulang"],
    'Perih Bak' : ["Penyakit Dalam", "Bedah", 'Urologi'],
    'Perih Kencing' : ["Penyakit Dalam", "Bedah", 'Urologi'],
    'Sakit Bak' : ["Penyakit Dalam", "Bedah", 'Urologi'],
    'Bak Sakit' : ["Penyakit Dalam", "Bedah", 'Urologi'],
    'Telat Datang Bulan'  : ["Kandungan"],
    'Imunitas'  : ['Gizi'],
    'Kencing Sedikit' : ['Urologi'],
    'Ulu Hati'  :   ['Bedah', 'Penyakit Dalam'],
    'Hulu Hati'  :   ['Bedah', 'Penyakit Dalam'],
    'Sakit Tel' : ['Gigi', 'THT'],
    'Punggung Sakit'  : ['Tulang', 'Saraf'],
    'Sakit Badan' : ['Tulang', 'Saraf'],
    'Linu'  : ['Tulang', 'Penyakit Dalam'],
    'Sakit Malu'  : ["Kulit dan Kelamin", 'Urologi'], # Sakit Kemaluan

    'Buah Zakar'  : ['Urologi', 'Kulit dan Kelamin'],
    'Bersin'  : ['THT'],
    'Suara Hilang'  : ['THT'],
    'Lulur'   : ['Kulit dan Kelamin'],
    'Kentut'  :   ['Penyakit Dalam'],
    'Urethral'  : ['Bedah', 'Penyakit Dalam'],
    'Sesak Dada'  : ['Paru'],
    'Hipokondria' : ['Jiwa'],
    'Dada Sakit'  : ['Paru', 'Jantung'],
    'Suntik Silikon'  : ['Kulit dan Kelamin'],
    'Berhubngan Suami' : ["Kandungan"],
    'Otot Lengan' : ['Tulang'],
    'Khayal'  : ['Jiwa'],
    'Datang Bulan'  : ['Kandungan'],
    'Lhran' : ['Kandungan'],  # Lahiran
    'Rproduksi' : ['Kandungan'],
    'Makan' : ['Gizi'],
    'Kelamin Lelaki'  : ['Kulit dan Kelamin', 'Urologi'],

    'Disorder'  : ['Jiwa'],
    'Micin Bodoh' : ['Gizi'],
    'Tinggi Badan'  : ['Gizi'],
    'Buka Mulut'  : ['Gigi'],
    'Fobia' : ['Jiwa'],
    'Rongga Mulut'  : ['Gigi'],
    'Cair Hidung' : ['THT'],
    'Badan Kurus' : ['Gizi'],
    'Sgot'  : ['Penyakit Dalam'],
    'Sgpt'  : ['Penyakit Dalam'],
    'Anosmia' : ['THT'],
    'Kalori'  : ['Gizi'],
    'Enzim Hati'  : ['Penyakit Dalam'],
    'Sel Telur' : ['Kandungan'],
    'Hitam Mulut' : ['Gigi'],
    'Hantu' : ['Jiwa'],
    'Sering Lupa'  : ['Saraf'],
    'Gusi Darah'  : ['Gigi'],
    'Rahang'  : ['Gigi'],
    'Firusa'  : ['Bedah'],        'Fisura'  : ['Bedah'],

    'Sulit Ingat' : ['Saraf'],
    "Bumbu Masak" : ['Gizi'],
    'Mancung Hidung'  : ['Bedah'],
    'Bindeng' : ['THT'],
    'Hilang Bulu' : ['Kulit dan Kelamin'],
    'Keloid'  : ['Bedah', 'Kulit dan Kelamin'],
    'Kb'  : ['Kandungan'],
    'Kandung Kemih' : ['Urologi'],
    'Neurogenik'  : ['Saraf'],
    'Janin' : ['Kandungan'],
    'Sulit Nafas' : ['THT'],
    'Hbv' : ['Penyakit Dalam'],
    'Kelamin' : ['Urologi', 'Kulit dan Kelamin'],
    'Mie Instan'  : ['Gizi'],
    'Salur Kemih' : ['Urologi'],
    'Sedih' : ['Jiwa'],
    'Kandung Kemih' : ['Urologi'],
    'Cholelithiasis'  : ['Bedah'],
    'Tubuh Kurus' : ['Gizi'],
    'bintik bintik' : ['Kulit dan Kelamin'],
    'nyeri' : ['Penyakit Dalam'],
    'mr p' : ['Kulit dan Kelamin'],
    'erythromycin' : ['Kulit dan Kelamin', 'Penyakit Dalam', 'Urologi'],
    'promag' : ['Penyakit Dalam'],
    'jari kaki kaku' : ['Penyakit Dalam'],
    'thorax' : ['Paru'],
    'dahak ada darah' : ['THT', 'Penyakit Dalam'],
    'celaka motor' : ['Penyakit Dalam', 'Tulang'],
    'insulin' : ['Penyakit Dalam'],
    'shabu' : ['Jiwa'],
    'filler hidung' : ['Kulit dan Kelamin'],
    'biasa onani' : ['Jiwa'],
    'ligamen' : ['Tulang'],
    'tendon' : ['Tulang'],
    'ganggu lihat' : ['Mata'],
    'ludah campur darah' : ['THT', 'Penyakit Dalam'],
    'tanam benang' : ['Kulit dan Kelamin'],
    'lepas iud' : ['Kandungan'],
    'Buduk' : ['Kulit dan Kelamin'],
    'cairanx pnuh v' : ["Kandungan"],
    'manusia paling dosa' : ["Jiwa"]

}

## Mapper

---



In [28]:
df_corpus["CLASS_JUDUL"] = [mapping_2(s) for s in df_corpus["JUDUL"]]
df_corpus["CLASS_ISI"] = [mapping_2(s) for s in df_corpus["ISI"]]
df_corpus["CLASS_JUDUL_ISI"] = [mapping_2((j + " SEP " + i)) for j, i in df_corpus[["JUDUL", "ISI"]].values]
df_corpus["NEW_CLASS"] = [mapping_2(" , ".join(s)) for s in df_corpus["CLASS"]]

In [29]:
def mapping_3(s):
  res = {}
  for key, val in map.items():
    if is_in(key.lower(), s.lower()):
      print(key)
      for label in val:
        if label not in res:
          res[label] = 0
        res[label] += 1
  return get_top_3(res), res

In [30]:
var = """Berkeringat saat tidur
Pagi dok saya mau tanya dok kenapa anak saya dari usia 1 tahun(sekarang sudah 4 th6 bln kalau tidur pasti berkeringat di tubuhnya? Satu lagi dok anak saya juga sering sekali demam dan batuk pilek.mohon jawabannya dok terima kasih.
"""
var = stemmer.stem(stopword.remove(re.sub(r'[^\w]', ' ', str(var).lower())))
var

'keringat tidur pagi dok mau tanya dok kenapa anak usia 1 tahun sekarang sudah 4 th6 bln kalau tidur pasti keringat tubuh satu lagi dok anak juga sekali demam batuk pilek mohon jawab dok terima kasih'

In [31]:
# var = "ibu saya Hematuria. ayah saya Kolesterol dan Nyeri Sendi. saya Sakit Bak dan Keloid beserta ludah campur darah"

# print(var.lower())
# var = stemmer.stem(stopword.remove(re.sub(r'[^\w]', ' ', str(var).lower())))
mapping_3(var)

Anak
Batuk
Demam
Keringat
Pilek


(['THT'], {'Anak': 1, 'THT': 2, 'Penyakit Dalam': 1, 'Kulit dan Kelamin': 1})

In [32]:
get_top_3({'Anak': 1,  'Penyakit Dalam': 1, 'Kulit dan Kelamin': 1, 'THT': 1})

['Anak', 'Penyakit Dalam', 'Kulit dan Kelamin']

In [33]:
df_corpus.head(3)

,CLASS,JUDUL,ISI,JAWABAN,CLASS_JUDUL,CLASS_ISI,CLASS_JUDUL_ISI,NEW_CLASS
ID,,,,,,,,
DS-1,[other],mengapa keringat badan lebih,tanya konsultasi selamat malam dokter fathurro...,selamat malam Fathurrosi. Maaf berapa berat ba...,(Kulit dan Kelamin),(Kulit dan Kelamin),(Kulit dan Kelamin),[]
DS-23,[other],suka menghayal halusinasi,tanya konsultasi selamat siang dokter kali kha...,"Bila berkelanjutan, bisa saja memiliki ganggua...",(Jiwa),(Jiwa),(Jiwa),[]
DS-87,[other],feses warna kuning serta darah apakah ada peny...,tanya konsultasi selamat sore dokter feses war...,Kemungkinan kamu mengalami wasir atau ambeien....,(Penyakit Dalam),(Penyakit Dalam),(Penyakit Dalam),[]


In [34]:
print("Jumlah instances CLASS_JUDUL yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL yang null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL"].str.len() == 0])))
print("Jumlah instances CLASS_ISI yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_ISI yang null: " + str(len(df_corpus[df_corpus["CLASS_ISI"].str.len() == 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL_ISI"].str.len() == 0])))

Jumlah instances CLASS_JUDUL yang bukan null: 1415
Jumlah instances CLASS_JUDUL yang null: 144
Jumlah instances CLASS_ISI yang bukan null: 1506
Jumlah instances CLASS_ISI yang null: 53
Jumlah instances CLASS_JUDUL_ISI yang bukan null: 1559
Jumlah instances CLASS_JUDUL_ISI yang null: 0


# Compare Metric

In [35]:
from sklearn.metrics import classification_report, accuracy_score, jaccard_score, \
hamming_loss, cohen_kappa_score, f1_score, precision_score, recall_score
import math
def eval_all(y_human, y_machine, mlb):
  print("================ Classification Report ================")
  print(classification_report(y_human, y_machine, target_names=mlb.classes_, digits=4))
  print("================ Multi Label Score ================")
  eval_multi_label(y_human, y_machine, mlb)
  print("================ Cohen Kappa ================")
  cohen_kappa(y_human, y_machine, mlb)

def eval_multi_label(y_test, prediction, mlb):
    # Butuh informasi lebih lengkap? silakan disimak di bawah ini
    print('Accuracy             :', accuracy_score(y_test, prediction))
    print('Precision            :', precision_score(y_test, prediction, average='micro'))
    print('Recall               :', recall_score(y_test, prediction, average='micro'))
    print('F1 Score             :', f1_score(y_test, prediction, average='micro'))
    print('Hamming Loss         :', hamming_loss(y_test, prediction))
    print('Jaccard Score')
    js = jaccard_score(y_test, prediction, average=None)
    for i, j in zip(mlb.classes_, js):
      print(f"{i:<20} : {j}")
    print('Jaccard Score Macro Average:', jaccard_score(y_test, prediction, average='macro'))
    print('Jaccard Score Micro Average:', jaccard_score(y_test, prediction, average='micro'))

def cohen_kappa(y_human, y_test, mlb):
  sum = 0
  count = 0
  for i in range(len(mlb.classes_)):
    score = cohen_kappa_score(y_human[:,i], y_test[:,i])
    if not math.isnan(score):
      sum += score
      count += 1
    print(f"Cohen Kappa {mlb.classes_[i]:<20} : {cohen_kappa_score(y_human[:,i], y_test[:,i])}")
  print("Average Cohen Kappa              : " + str(sum/count))

## By Metadata

In [36]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_ISI", "CLASS_JUDUL_ISI", "CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
class_col = "NEW_CLASS"
df = df[df['NEW_CLASS'].map(lambda d: len(d)) > 0]
df

,NEW_CLASS,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
AD-27,(Gizi),0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
AD-47,(Kandungan),0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
AD-117,"[Bedah, Penyakit Dalam]",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
AD-148,[Kulit dan Kelamin],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
AD-179,[Kandungan],0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43177,(Penyakit Dalam),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,(THT),0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
KD-43201,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb_std = MultiLabelBinarizer()
mlb_std.fit(df[class_col].values)

MultiLabelBinarizer()

In [38]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [39]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.8160    0.7846    0.8000       130
            Bedah     0.4593    0.5299    0.4921       117
             Gigi     0.9355    0.8529    0.8923        34
             Gizi     0.7403    0.5534    0.6333       103
          Jantung     0.1852    0.2273    0.2041        22
             Jiwa     0.8442    0.7471    0.7927        87
        Kandungan     0.8811    0.8163    0.8475       245
Kulit dan Kelamin     0.8079    0.8363    0.8218       171
             Mata     1.0000    0.8841    0.9385        69
             Paru     0.5625    0.5000    0.5294        18
   Penyakit Dalam     0.5468    0.7264    0.6239       201
            Saraf     0.4138    0.2449    0.3077        49
              THT     0.9091    0.8029    0.8527       137
           Tulang     0.6667    0.5714    0.6154        42
          Urologi     0.5000    0.0606    0.1081        33

## By Judul

In [40]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_ISI", "CLASS_JUDUL_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_JUDUL"].str.len() > 0]
class_col = "CLASS_JUDUL"
df

,CLASS_JUDUL,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,(Kulit dan Kelamin),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-23,(Jiwa),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
DS-87,(Penyakit Dalam),0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
DS-176,"[Bedah, Penyakit Dalam]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-324,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43177,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,(THT),0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
KD-43201,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [42]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.7432    0.6111    0.6707       180
            Bedah     0.4724    0.6861    0.5595       137
             Gigi     0.9167    0.9429    0.9296        35
             Gizi     0.4300    0.8558    0.5723       104
          Jantung     0.4375    0.7778    0.5600        36
             Jiwa     0.9500    0.6264    0.7550        91
        Kandungan     0.6863    0.8185    0.7466       270
Kulit dan Kelamin     0.6920    0.8693    0.7706       199
             Mata     0.9359    0.9733    0.9542        75
             Paru     0.3636    0.5000    0.4211        16
   Penyakit Dalam     0.5336    0.6368    0.5806       212
            Saraf     0.4694    0.4600    0.4646        50
              THT     0.8978    0.8483    0.8723       145
           Tulang     0.7027    0.6341    0.6667        41
          Urologi     0.5357    0.4412    0.4839        34

## By Isi

In [43]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_JUDUL_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_ISI"].str.len() > 0]
class_col = "CLASS_ISI"
df

,CLASS_ISI,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,(Kulit dan Kelamin),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-23,(Jiwa),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
DS-87,(Penyakit Dalam),0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
DS-176,"[Bedah, Penyakit Dalam]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-277,"[Saraf, Jiwa, Paru]",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43177,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,"[Jantung, THT]",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
KD-43201,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [45]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.6855    0.6022    0.6412       181
            Bedah     0.4513    0.6197    0.5223       142
             Gigi     0.8293    0.9444    0.8831        36
             Gizi     0.3371    0.8241    0.4785       108
          Jantung     0.3452    0.7838    0.4793        37
             Jiwa     0.8857    0.5849    0.7045       106
        Kandungan     0.6776    0.8350    0.7481       297
Kulit dan Kelamin     0.7377    0.8867    0.8054       203
             Mata     0.8293    0.8831    0.8553        77
             Paru     0.3200    0.4000    0.3556        20
   Penyakit Dalam     0.4858    0.6444    0.5540       239
            Saraf     0.3385    0.3607    0.3492        61
              THT     0.8467    0.8411    0.8439       151
           Tulang     0.5385    0.6087    0.5714        46
          Urologi     0.4400    0.2973    0.3548        37

## By Judul + Isi

In [46]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_JUDUL_ISI"].str.len() > 0]
class_col = "CLASS_JUDUL_ISI"
df

,CLASS_JUDUL_ISI,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,(Kulit dan Kelamin),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-23,(Jiwa),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
DS-87,(Penyakit Dalam),0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
DS-176,"[Bedah, Penyakit Dalam]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-277,"[Saraf, Jiwa, Paru]",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43177,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,[THT],0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
KD-43201,(Bedah),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [48]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.7006    0.6324    0.6648       185
            Bedah     0.4559    0.6414    0.5330       145
             Gigi     0.8140    0.9459    0.8750        37
             Gizi     0.3596    0.8649    0.5079       111
          Jantung     0.3721    0.8421    0.5161        38
             Jiwa     0.8846    0.6161    0.7263       112
        Kandungan     0.6921    0.8328    0.7560       305
Kulit dan Kelamin     0.7433    0.9023    0.8151       215
             Mata     0.8571    0.9000    0.8780        80
             Paru     0.3636    0.3810    0.3721        21
   Penyakit Dalam     0.4865    0.6585    0.5596       246
            Saraf     0.3548    0.3492    0.3520        63
              THT     0.8618    0.8397    0.8506       156
           Tulang     0.5370    0.6042    0.5686        48
          Urologi     0.5000    0.3333    0.4000        39

# Prediksi

In [49]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import re
import random
import seaborn as sns
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.probability import FreqDist
nltk.download('punkt')

from xgboost import XGBClassifier

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score, hamming_loss

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [50]:
seed_val = 1906350912
random.seed(seed_val)
np.random.seed(seed_val)
categories = ['Anak', 'Bedah', 'Gigi', 'Gizi', 'Jantung', 'Jiwa',
       'Kandungan', 'Kulit dan Kelamin', 'Mata', 'Paru', 'Penyakit Dalam',
       'Saraf', 'THT', 'Tulang', 'Urologi']

categories_umum = ['Anak', 'Bedah', 'Gigi', 'Gizi', 'Jantung', 'Jiwa',
       'Kandungan', 'Kulit dan Kelamin', 'Mata', 'Paru', 'Penyakit Dalam',
       'Saraf', 'THT', 'Tulang', 'Umum', 'Urologi']
# torch.manual_seed(seed_val)

In [51]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit_transform([categories])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [52]:
df_test = pd.read_csv(f"Dataset/Gold_Standard.csv", index_col="ID")
df_test.fillna("", inplace = True)
df_test['Count'] = df_test.drop(columns=['JUDUL', 'ISI']).values.sum(axis=1)
df_test = df_test[df_test['Count'] <= 3].drop(columns=['Count'])
df_test = df_test[["JUDUL", "ISI"] + categories_umum]

df_test_umumnt = df_test.drop(columns=["Umum"])
df_test_umumnt["COUNT CLASS"] = df_test_umumnt.drop(columns=["JUDUL", "ISI"]).sum(axis=1)
df_test = df_test_umumnt[df_test_umumnt["COUNT CLASS"] > 0].drop(columns=["COUNT CLASS"])
del df_test_umumnt

In [53]:
import re
df_test["JUDUL"] = [re.sub(r'[^\w]', ' ', str(s).lower()) for s in df_test["JUDUL"].values]
df_test["ISI"] = [re.sub(r'[^\w]', ' ', str(s).lower()) for s in df_test["ISI"].values]

In [54]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
factory = StopWordRemoverFactory()
# stopwords = factory.get_stop_words()
# ext_stopwords = ['dok', 'doc', 'dokter', 'terima', 'kasih', 'terimakasih']
# whitelist = ['kecil', 'besar', 'mata', 'masalah', 'belakang']
# all_stopwords = list((set(stopwords) | set(ext_stopwords)) - set(whitelist))
all_stopwords = ["di", 'pada', 'ingin', 'dan', 'yang', 'sangat', 'luar', 'biasa', 'bagi', 'bagian', 'saat', 'hanya', 'asa', 'terasa', 'saat', 'dengan', 'bikin', 'dari', 'dalam', 'sering', 'saya', 'untuk']
dictionary = ArrayDictionary(all_stopwords)
stopword = StopWordRemover(dictionary)
df_test["JUDUL"] = [stopword.remove(s) for s in df_test["JUDUL"]]
df_test["ISI"] = [stopword.remove(s) for s in df_test["ISI"]]

In [55]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
ignore = ["menyusui"]
# [" ".join([stemmer.stem(w) if w not in ignore else w for w in s.split()]) for s in lst])]
df_test["JUDUL"] = [" ".join([stemmer.stem(w) if w not in ignore else w for w in str(s).split()]) for s in df_test["JUDUL"]]
df_test["ISI"] = [" ".join([stemmer.stem(w) if w not in ignore else w for w in str(s).split()]) for s in df_test["ISI"]]

In [56]:
df_test["CLASS_JUDUL"] = [mapping_2(s) for s in df_test["JUDUL"]]
df_test["CLASS_ISI"] = [mapping_2(s) for s in df_test["ISI"]]
df_test["CLASS_JUDUL_ISI"] = [mapping_2((j + " SEP " + i)) for j, i in df_test[["JUDUL", "ISI"]].values]

In [57]:
print("Jumlah instances CLASS_JUDUL yang bukan null: " + str(len(df_test[df_test["CLASS_JUDUL"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL yang null: " + str(len(df_test[df_test["CLASS_JUDUL"].str.len() == 0])))
print("Jumlah instances CLASS_ISI yang bukan null: " + str(len(df_test[df_test["CLASS_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_ISI yang null: " + str(len(df_test[df_test["CLASS_ISI"].str.len() == 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang bukan null: " + str(len(df_test[df_test["CLASS_JUDUL_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang null: " + str(len(df_test[df_test["CLASS_JUDUL_ISI"].str.len() == 0])))

Jumlah instances CLASS_JUDUL yang bukan null: 411
Jumlah instances CLASS_JUDUL yang null: 58
Jumlah instances CLASS_ISI yang bukan null: 449
Jumlah instances CLASS_ISI yang null: 20
Jumlah instances CLASS_JUDUL_ISI yang bukan null: 464
Jumlah instances CLASS_JUDUL_ISI yang null: 5


In [58]:
y_pred = mlb.transform(df_test["CLASS_JUDUL"].values)

In [59]:
y_test = df_test[categories].values

In [60]:
len(df_test)

469

In [61]:
eval_all(y_test, y_pred, mlb)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.9394    0.7209    0.8158        43
            Bedah     0.4630    0.8621    0.6024        29
             Gigi     1.0000    0.9286    0.9630        42
             Gizi     0.8333    0.9302    0.8791        43
          Jantung     0.5333    0.8000    0.6400        10
             Jiwa     1.0000    0.6512    0.7887        43
        Kandungan     0.8108    0.6977    0.7500        43
Kulit dan Kelamin     0.9111    0.8913    0.9011        46
             Mata     1.0000    0.9231    0.9600        26
             Paru     0.3333    0.5000    0.4000         2
   Penyakit Dalam     0.5152    0.8095    0.6296        42
            Saraf     0.5000    0.3571    0.4167        14
              THT     0.9722    0.8140    0.8861        43
           Tulang     0.9355    0.8286    0.8788        35
          Urologi     0.8000    0.2857    0.4211        14

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro', zero_division=0)
hamming = hamming_loss(y_test, y_pred)

print(f'Accuracy: {acc}')
print(f'F1 (Micro): {f1}')
print(f'Hamming Loss: {hamming}')
print(classification_report(y_test, y_pred, target_names=categories, zero_division=0))

Accuracy: 0.6886993603411514
F1 (Micro): 0.7881981032665965
Hamming Loss: 0.02857142857142857
                   precision    recall  f1-score   support

             Anak       0.94      0.72      0.82        43
            Bedah       0.46      0.86      0.60        29
             Gigi       1.00      0.93      0.96        42
             Gizi       0.83      0.93      0.88        43
          Jantung       0.53      0.80      0.64        10
             Jiwa       1.00      0.65      0.79        43
        Kandungan       0.81      0.70      0.75        43
Kulit dan Kelamin       0.91      0.89      0.90        46
             Mata       1.00      0.92      0.96        26
             Paru       0.33      0.50      0.40         2
   Penyakit Dalam       0.52      0.81      0.63        42
            Saraf       0.50      0.36      0.42        14
              THT       0.97      0.81      0.89        43
           Tulang       0.94      0.83      0.88        35
          Urologi   